<a href="https://colab.research.google.com/github/adhiutama17/MKBigData/blob/main/FixTugas_TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fix untuk Tugas MK Big Data - Text Summarization

## Daftar Isi



*   Pendahuluan
*   Teori
*   Metode
*   Uji Coba
*   Validasi
*   Test



##Pendahuluan
Era Internet menyediakan beragam informasi yang bisa diakses kapan pun dan dimana pun, misalnya berita elektronik, iklan, kursus online, hingga artikel-artikel ilmiah. Contoh terakhir menjadi salah satu sumber informasi yang wajib di dunia pendidikan khususnya pendidikan tinggi. Artikel ilmiah didefinisikan sebagai karya ilmu pengetahuan yang menyajikan fakta umum dan ditulis dengan metodologi penulisan yang baik dan benar. Konten dari artikel ilmiah dapat berupa penelitian, ulasan teori, hingga ide baru seseorang didasarkan suatu permasalahan. Menurut World Bank , dari tahun 2000 hingga 2018 tercatat lebih dari 2,5 juta artikel ilmiah yang terpublikasi melalui berbagai macam lembaga publikasi dengan pengelompokkan bidang tertentu. Pembaca artikel ilmiah perlu menemukan informasi dan gagasan penulis dari teks yang ditulis. Selain artikel ilmiah yang terpublikasi, terdapat juga artikel yang belum terpublikasi berupa paper, jurnal, atau artikel esai perlombaan mahasiswa.

#Dasar Teori

###Text Summarization

Text Summarization (Ringkasan Teks) merupakan teks yang terdiri atas satu atau dua kalimat yang menyampaikan informasi penting dari teks yang asli, biasanya jumlah kalimat tidak lebih dari setengah teks yang asli bahkan lebih sedikit

*   Extractive

mengidentifikasi bagian penting dari teks dan ditulis kembali dalam teks yang ringkas. Pendekatan ini akan bergantung pada kalimat-kalimat yang tercantum pada teks tersebut

*   Abstractive

mengidentifikasi dan mengintepretasikan bagian penting dari teks dengan membuat kalimat yang baru. Kalimat baru dari pendekatan ini dapat dimungkinkan tidak menggunakan kalimat yang tercantum tetapi yang lain dengan makna yang sama



#Repository File
Tools pembuatan program ini menggunakan python dengan menghubungan repository Github

In [ ]:
!git clone https://github.com/adhiutama17/MKBigData

In [ ]:
!ls MKBigData

In [ ]:
#File Path untuk Teks yang akan diolah
#Contoh file yang akan diolah telah ditempatkan di folder MKBigData/Teks/
pdf_path = 'MKBigData/Teks/Abdul Rohman_Universitas Airlangga_Provider R_D.pdf' #file dalam bentuk PDF maka diperlukan tools pdfminer

#Pre-Process - membuka file pdf
file pdf di github akan dibuka dengan tools pdfminer.six

In [ ]:
pip install pdfminer.six

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import resolve1
from pdfminer.high_level import extract_text

file = open(pdf_path, 'rb') #membuka file pdf
parser = PDFParser(file)
document = PDFDocument(parser)

print(resolve1(document.catalog['Pages'])['Count']) #mendapatkan jumlah halaman, karena terdapat halaman yang tidak diperlukan nantinya

In [ ]:
def load_data(pdf_path, hal_awal, hal_akhir):
    """
    Membuat fungsi def untuk memanggil, membuka file pdf
    Input  : pdf_path (lokasi file), halaman awal (bukan cover) dan akhir (sebelum daftar pustaka)
    Output : kumpulan kalimat (disebut juga document)
    """
    file_pdf = open(pdf_path, 'rb') #membuka file pdf
    parser = PDFParser(file_pdf)
    document = PDFDocument(parser)
    file_text = extract_text(pdf_path, page_numbers = range(hal_awal,hal_akhir))
    text_split = file_text.split(". ")

    print('Jumlah Halaman: '+resolve1(document.catalog['Pages'])['Count']) #mendapatkan jumlah halaman, karena terdapat halaman yang tidak diperlukan nantinya
    print(text_split)

#Pre-Process - Lower, Stopword Remove dan Stemming


*   setiap kata dalam dokumen akan dikonversi ke huruf kecil
*   Menghapus stopword berbahasa Indonesia (menggunakan library sastrawi)
*   Stemming (kata dasar), konversi setiap kata menjadi kata dasar



In [ ]:
#Menggunakan tools yang diperlukan
import nltk
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
#Membuat list stopword berbahasa Indonesia
stop_factory = StopWordRemoverFactory()
#menambahkan stopword yang didapatkan dari library lainnya
tambah_stopword = open('MKBigData/stopword-indo.txt',"r").readlines()
tambah_stopword_clr = [item.replace("\n", "") for item in tambah_stopword]
data_stop = stop_factory.get_stop_words() + tambah_stopword_clr
stopword = stop_factory.create_stop_word_remover()

In [ ]:
def preprocess_data(doc_set, stopword_list):
    """Input  : dokumen input
    Purpose: preprocess text (tokenize, removing stopwords, and stemming)
    Output : korpus/ teks bersih """
    #Membuat Stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    tokenizer = RegexpTokenizer(r'\w+')
    texts = []
    for i in doc_set:
      #Tokenisasi kalimat menjadi kata
      raw = i.lower()
      tokens = tokenizer.tokenize(raw)
      #Menghapus stopword
      stopped_tokens = [i for i in tokens if not i in stopword_list]
      #stem tokens
      stemmed_tokens = [stemmer.stem(i) for i in stopped_tokens]
      #list token
      texts.append(stemmed_tokens)
    return texts

#Teori Latent Semantic Analysis
Latent Semantic Analysis mengkoversi kumpulan kata ke dalam matriks term-document, yakni setiap baris mewakili kata-kata unik, sedangkan kolom mewakili konteks darimana kata-kata tersebut diambil

Mempersiapkan korpus dari teks bersih untuk di-konversi menjadi doc-term matrix

In [ ]:
def prepare_corpus(doc_clean):
    """
    Input  : teks yang sudah melalui pre-process
    Purpose: membuat doc-term matrix dari korpus (teks bersih)
    Output : term dictionary and Document Term Matrix
    """
    # Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
    dictionary = corpora.Dictionary(doc_clean)
    # Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
    doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
    # generate LDA model
    return dictionary,doc_term_matrix

Pemodelan LSA

In [ ]:
def create_gensim_lsa_model(doc_clean,number_of_topics,words):
    """
    Input  : korpus/ teks bersih, jumlah topik yg dijadikan acuan, jumlah kata yg berkaitan dgn topik
    Purpose: membuat model LSA
    Output : model LSA
    """
    dictionary,doc_term_matrix=prepare_corpus(doc_clean)
    # membuat model LSA
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(lsamodel.print_topics(num_topics=number_of_topics, num_words=words))
    return lsamodel

In [ ]:
def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop, start=2, step=3):
    """
    Input   : dictionary : Gensim dictionary
              corpus : Gensim corpus
              texts : List of input texts
              stop : Max num of topics
    purpose : Compute c_v coherence for various number of topics
    Output  : model_list : List of LSA topic models
              coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values